# INFORMACIÓN DEL PROYECTO

### TÍTULO

Ucrania: pérdidas rusas y el coste de la comida

### TEMA

Una comparativa de 5 datasets de tamaños variables sobre diversos aspectos la guerra de Ucrania que tienen que ver con las pérdidas en el bando ruso, los refugiados y el precio de la comida en los puestos locales. El nexo de la mayoría son las fechas.

### HIPÓTESIS

Define aquí lo que será la hipótesis de tu proyecto.
Puede que tengas más de una, pero tu proyecto lo harás teniendo una principal hipótesis.

1) Se espera que el precio de la comida en Ucrania oscile en función del número de refugiados que huyen del país y de la cercanía al frente de los puestos de venta. Asimismo, un peor rendimiento por parte del bando ruso debería animar a los proveedores a vender en el país y aumentar la oferta.

Más refugiados, mayor lejanía del frente, más pérdidas rusas = menos precios

2) Con el paso del tiempo, aumentan las pérdidas materiales rusas y se reducen las pérdidas humanas. Ello puede ser indicativo de una creciente preocupación de la administración rusa por la opinión pública, con lo que, entre otras cosas, usa más drones o artillería y menos tropas terrestres para minimizar la cuenta de bajas.

Más pérdidas materiales = menos muertos = uso reducido de soldados rasos = uso mayor de tropas mecanizadas

3) El ejército ruso todavía se sirve de mucho material de la Unión Soviética, que, en comparación con el equipamiento moderno, tiende a ser capturado o destruido con mayor frecuencia.

Uso de material soviético = más destrucciones = más capturas

## OBTENCIÓN DE LOS DATOS

### DATASETS Y FUENTES ALTERNATIVAS DE DATOS

Incluye aquí una breve descripción del dataset o datasets de los que partirás para poder evaluar tu hipótesis. <br>
También incluye el origen de estos datos y su fuente.

Refugiados ucranianos (acumulativo) por fecha y destino:
https://www.kaggle.com/datasets/anuragbantu/ukraine-invasion-refugee-data-2022

Precio de la comida en cada mercado de Ucrania por fecha y tipo. He añadido una columna y he apuntado, a mano, en base a un mapa del país, la distancia aproximada de cada ciudad respecto a la frontera este, que es donde se abre el frente:
https://data.humdata.org/dataset/wfp-food-prices-for-ukraine

Pérdidas rusas personales por fecha:
https://www.kaggle.com/datasets/piterfm/2022-ukraine-russian-war

Pérdidas rusas materiales, según categorías generales, por fecha (mismo Kaggle que el anterior):
https://www.kaggle.com/datasets/piterfm/2022-ukraine-russian-war

Pérdidas materiales rusas (incluye modelo y fabricante), por fecha:
https://www.kaggle.com/datasets/piterfm/2022-ukraine-russia-war-equipment-losses-oryx


Muestra mediante un head() los principales datasets con los que vas a trabajar

In [1]:
# Refugiados ucranianos (acumulativo) por fecha y destino:

import pandas as pd

df_refugees = pd.read_csv(r'src\data\Refugees.csv')

df_refugees = df_refugees[['date', 'country', 'individuals']]

df_refugees.head()


,date,country,individuals
0,2022-09-13,Poland,6114787
1,2022-09-12,Russian Federation,2593209
2,2022-09-13,Slovakia,778335
3,2022-09-10,Republic of Moldova,606707
4,2022-09-11,Belarus,16705


In [2]:
# Precio de la comida en cada mercado de Ucrania por fecha y tipo + columna de distancias aproximadas de la frontera este:

import datetime

df_prices = pd.read_csv(r'src\data\Prices.csv', low_memory=False)

df_prices = df_prices.drop(index=0)

df_prices['date'] = pd.to_datetime(df_prices['date'], exact=False).dt.date

start_war = datetime.date(2022, 2, 24)

df_prices = df_prices[df_prices['date'] >= start_war]

df_prices.reset_index(drop=True, inplace=True)

df_prices = df_prices[['date', 'category', 'commodity', 'unit', 'usdprice', 'market']]

distance_to_east = {'Kherson': 'very close',
                    'Dnipropetrovsk': 'close',
                    'Vinnytsya': 'medium',
                    'Zhytomyr': 'medium-far',
                    'Poltava': 'medium-close',
                    'Mykolayiv': 'close',
                    'Kyiv city': 'medium',
                    'Iv.-Frankivsk': 'far',
                    'Volyn': 'very far',
                    'Zaporizhzhya': 'very close',
                    'Khmelnytskiy': 'medium-far',
                    'Chernivtsi': 'far',
                    'Kirovohrad': 'close',
                    'Lviv': 'very far',
                    'Zakarpattya': 'very far',
                    'Sumy': 'medium-close',
                    'Ternopyl': 'far',
                    'Odesa': 'close',
                    'Cherkasy': 'medium-close',
                    'Kyiv': 'medium',
                    'Rivne': 'far',
                    'Chernihiv': 'medium',
                    'Kharkiv': 'close',
                    'Donetsk': 'very close'
                }

for key, value in distance_to_east.items():
    df_prices.loc[df_prices['market'] == key, 'to eastern frontier'] = value

df_prices.head()


,date,category,commodity,unit,usdprice,market,to eastern frontier
0,2022-03-15,cereals and tubers,Barley,KG,0.502,Cherkasy,medium-close
1,2022-03-15,cereals and tubers,Bread (first grade flour),KG,0.7094,Cherkasy,medium-close
2,2022-03-15,cereals and tubers,Bread (high grade flour),KG,0.8548,Cherkasy,medium-close
3,2022-03-15,cereals and tubers,Bread (rye),KG,0.7021,Cherkasy,medium-close
4,2022-03-15,cereals and tubers,Bread (wheat),500 G,0.4372,Cherkasy,medium-close


In [3]:
# Pérdidas rusas personales por fecha:

df_personnel = pd.read_csv(r'src\data\Russia_losses_personnel(date).csv')

for index, casualties in enumerate(df_personnel['personnel']):
    if index != 0:
        previous_index = index - 1
        df_personnel.loc[[index], 'absolute personnel'] = int(df_personnel.loc[[index], 'personnel']) - int(df_personnel.loc[[previous_index], 'personnel'])
    else:
         df_personnel.loc[[index], 'absolute personnel'] = df_personnel.loc[[index], 'personnel']

df_personnel = df_personnel[['date', 'absolute personnel', 'personnel']]

df_personnel.head()


,date,absolute personnel,personnel
0,2022-02-25,2800.0,2800
1,2022-02-26,1500.0,4300
2,2022-02-27,200.0,4500
3,2022-02-28,800.0,5300
4,2022-03-01,410.0,5710


In [4]:
# Pérdidas rusas materiales, según categorías generales, por fecha:

df_equipment = pd.read_csv(r'src\data\Russia_losses_equipment(date).csv')

df_equipment.drop(columns=['day', 'greatest losses direction'], inplace=True)

df_equipment.head()


,date,aircraft,helicopter,tank,APC,field artillery,MRL,military auto,fuel tank,drone,naval ship,anti-aircraft warfare,special equipment,mobile SRBM system,vehicles and fuel tanks,cruise missiles
0,2022-02-25,10,7,80,516,49,4,100.0,60.0,0,2,0,NaN,NaN,NaN,NaN
1,2022-02-26,27,26,146,706,49,4,130.0,60.0,2,2,0,NaN,NaN,NaN,NaN
2,2022-02-27,27,26,150,706,50,4,130.0,60.0,2,2,0,NaN,NaN,NaN,NaN
3,2022-02-28,29,29,150,816,74,21,291.0,60.0,3,2,5,NaN,NaN,NaN,NaN
4,2022-03-01,29,29,198,846,77,24,305.0,60.0,3,2,7,NaN,NaN,NaN,NaN


In [5]:
# Pérdidas materiales rusas (incluye modelo y fabricante), por fecha:

df_tech = pd.read_csv(r'src\data\Russia_losses_equipment(tech_details).csv')

for column in df_tech.columns[4:]:
    pd.to_numeric(df_tech[column])

df_tech['total captured'] = df_tech[['captured', 
                                    'captured and destroyed',
                                    'captured and stripped',
                                    'damaged and captured'
                                    ]].sum(axis=1)

df_tech['total not captured'] = df_tech[['abandoned', 
                                        'abandoned and destroyed',
                                        'damaged',
                                        'damaged and abandoned',
                                        'damaged beyond economical repair',
                                        'damaged by Bayraktar TB2',
                                        'destroyed',
                                        'destroyed by Bayraktar TB2',
                                        'destroyed by Bayraktar TB2 and Harpoon AShM',
                                        'sunk'
                                        ]].sum(axis=1)

df_tech = df_tech[['equipment', 'model', 'manufacturer', 'losses_total', 'total captured', 'total not captured']]

df_tech.head()


,equipment,model,manufacturer,losses_total,total captured,total not captured
0,Tanks,T-62M,the Soviet Union,20,16.0,4.0
1,Tanks,T-62MV,the Soviet Union,3,2.0,1.0
2,Tanks,T-64A,the Soviet Union,2,0.0,2.0
3,Tanks,T-64BV,the Soviet Union,39,5.0,34.0
4,Tanks,T-72A,the Soviet Union,33,15.0,18.0
